In [130]:
import pandas as pd
import numpy as np
import igraph as ig
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, precision_score, recall_score
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
import igraph as ig
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import deque
import random
from sklearn.metrics import f1_score, precision_score, recall_score
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx
from torch.nn import Linear



In [ ]:
classes_path = "../elliptic_bitcoin_dataset/modified_elliptic_txs_classes.csv"
edges_path = "../elliptic_bitcoin_dataset/modified_elliptic_txs_edgelist.csv"
features_path = "../elliptic_bitcoin_dataset/modified_elliptic_txs_features.csv"

classes = pd.read_csv(classes_path)
edges = pd.read_csv(edges_path)
feat_cols = ['txId', 'time_step'] + [f'trans_feat_{i}' for i in range(93)] + [f'agg_feat_{i}' for i in range(72)]
feats = pd.read_csv(features_path, header=None, names=feat_cols)

# Preprocess the classes DataFrame
classes.columns = ['txId', 'label']
df = classes.set_index('txId').join(feats.set_index('txId'))

# Filter out 'unknown' labels
df_filtered = df[df['label'] != 'unknown']

# Convert node features and labels
node_features = torch.tensor(df_filtered.iloc[:, 1:].values, dtype=torch.float)
filtered_edges_list = [
    (edges[edges['txId1'][i]], edges[edges['txId2'][i]])
    for i in range(len(edges))
    if edges['txId1'][i] in df_filtered['txId'].values and 
       edges['txId2'][i] in df_filtered['txId'].values
]
filtered_node_features = df_filtered.drop(columns=['label', 'time_step']).values
filtered_labels = np.array([label_mapping[label] for label in df_filtered['label']])
filtered_edges_dict = {tx_id: i for i, tx_id in enumerate(classes['txId'])}
filtered_edge_index = torch.tensor([
    (filtered_edges_dict[tx1], filtered_edges_dict[tx2])
    for tx1, tx2 in filtered_edges_list
], dtype=torch.long).T
filtered_data = Data(
    x=torch.tensor(filtered_node_features, dtype=torch.float),
    edge_index=filtered_edge_index,
    y=torch.tensor(filtered_labels, dtype=torch.long)
)

edge_index = torch.tensor(filtered_edges_dict.values.T, dtype=torch.int64)
label_mapping = {'unknown': 0,'class1': 1, 'class2': 2} 
labels = torch.tensor(df['label'].map(label_mapping).values, dtype=torch.long)

# Create graph data object
data = Data(x=node_features, edge_index=edge_index, y=labels)
print(node_features)
edges = pd.read_csv(edges_path)

# Check the maximum index in the edges
print(f"Max edge index: {edges.max().max()}")
print(f"Number of nodes: {len(node_features)}")


KeyError: 'txId'

In [ ]:
# Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.gcn = GCNConv(num_features, 3)  # GCNConv layer, 3 output channels
        self.out = Linear(3, num_classes)    # Linear layer for classification output
    
    def forward(self, x, edge_index):
        h = self.gcn(x, edge_index).relu()   # Apply GCN and ReLU
        z = self.out(h)                      # Output layer
        return h, z

# Initialize the model
num_features = data.x.shape[1]  # Number of features (columns in x)
num_classes = len(label_mapping)  # Number of classes (2 in this case)
model = GCN(num_features, num_classes)
print(model)

# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)

# Accuracy calculation function
def accuracy(pred_y, y):
    return (pred_y == y).sum() / len(y)

# Training loop
for epoch in range(201):
    optimizer.zero_grad()
    h, z = model(data.x, data.edge_index)  # h: embeddings, z: logits
    loss = criterion(z, data.y)            # Compute loss
    loss.backward()                        # Backpropagate
    optimizer.step()                       # Update model parameters
    
    if epoch % 10 == 0:
        acc = accuracy(z.argmax(dim=1), data.y)  # Calculate accuracy
        print(f'Epoch {epoch:>3} | Loss: {loss:.2f} | Acc: {acc*100:.2f}%')

GCN(
  (gcn): GCNConv(166, 3)
  (out): Linear(in_features=3, out_features=3, bias=True)
)


IndexError: Target -9223372036854775808 is out of bounds.

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn = GCNConv(166, 3)
        self.out = Linear(3, 2)
    def forward(self, x, edge_index):
        h = self.gcn(x, edge_index).relu()
        z = self.out(h)
        return h, z
model = GCN()
print(model)

GCN(
  (gcn): GCNConv(166, 3)
  (out): Linear(in_features=3, out_features=2, bias=True)
)


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)

# Calculate accuracy
def accuracy(pred_y, y):
    return (pred_y == y).sum() / len(y)
# Training loop
for epoch in range(1):
    optimizer.zero_grad()
    h, z = model(data.x, data.edge_index)  # h: embeddings, z: logits
    loss = criterion(z, data.y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        acc = accuracy(z.argmax(dim=1), data.y)
        print(f'Epoch {epoch:>3} | Loss: {loss:.2f} | Acc: {acc*100:.2f}%')

RuntimeError: index 5530458 is out of bounds for dimension 0 with size 46564

In [ ]:
print(h.shape)
# Estrazione degli embeddings dopo aver addestrato la GCN
model.eval()
with torch.no_grad():
    embeddings, _ = model(data.x, data.edge_index)  # h: embeddings

# Converti gli embeddings in un DataFrame Pandas
embeddings_df = pd.DataFrame(embeddings.cpu().numpy())

# Salva gli embeddings su un file CSV
embeddings_df.to_csv('embeddings.csv', index=False)